In [4]:
import scipp as sc
import scippneutron as scn
import numpy as np

In [5]:
scn.__version__

'0.4.2'

In [6]:
path = 'Larmor_data'
data = scn.load_nexus(data_file=f'{path}/2021-02-23_0939_with_monitors.nxs')

/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess/lib/python3.8/site-packages/scippneutron/file_loading/_monitor_data.py:31: UserWarning: Event data present in NXMonitor group /entry/instrument/monitor_1. Histogram-mode monitor data from this group will be ignored.
  warnings.warn(f"Event data present in NXMonitor group {group.name}. "
/Users/wojciechpotrzebowski/opt/anaconda3/envs/ess/lib/python3.8/site-packages/scippneutron/file_loading/_monitor_data.py:31: UserWarning: Event data present in NXMonitor group /entry/instrument/monitor_2. Histogram-mode monitor data from this group will be ignored.
  warnings.warn(f"Event data present in NXMonitor group {group.name}. "


In [7]:
data

<scipp.DataArray>
Dimensions: Sizes[detector_id:401408, tof:1, ]
Coordinates:
  detector_id                 int32  [dimensionless]  (detector_id)  [1, 2, ..., 401407, 401408]
  position                  vector3              [m]  (detector_id)  [(0.499023, 0.41833, -4.42986), (0.500969, 0.41833, -4.42986), ..., (0.993148, 1.97683, -4.35336), (0.995093, 1.97683, -4.35336)]
  source_position           vector3              [m]  ()  [(0, 0, 25.3)]
  tof                         int32             [ns]  (tof [bin-edge])  [33892, -2147483648]
Data:
                          DataArrayView  [dimensionless]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

]
Attributes:
  end_time                   string  [dimensionless]  ()  ["2021-02-23T08:40:00.000Z"]
  experiment_title           string  [dimensionless]  ()  ["My experiment"]
  monitor_1               DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:118013, tof:1, ]
Coordinates:
  detector_id                 int32  [dimensionless]  (detector_id)  [21, 22, ..., 458737, 458739]
  tof                         int32             [ns]  (tof [bin-edge])  [33892, -2147483648]
Data:
                          DataArrayView  [dimensionless]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:1, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2021-02-23T09:39:13.789775236]
  tof                         int32             [ns]  (event)  [1337174827]
Data:
                            float32         [counts]  (event)  [1]  [1]

, <scipp.DataArray>
Dimensions: Sizes[event:1, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2021-02-23T09:39:13.789775236]
  tof                         int32             [ns]  (event)  [1521115875]
Data:
                            float32         [counts]  (event)  [1]  [1]

, ..., <scipp.DataArray>
Dimensions: Sizes[event:1, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2021-02-23T09:39:13.789775236]
  tof                         int32             [ns]  (event)  [2147483647]
Data:
                            float32         [counts]  (event)  [1]  [1]

, <scipp.DataArray>
Dimensions: Sizes[event:1, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2021-02-23T09:39:13.789775236]
  tof                         int32             [ns]  (event)  [178208161]
Data:
                            float32         [counts]  (event)  [1]  [1]

]

]
  monitor_2               DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:118013, tof:1, ]
Coordinates:
  detector_id                 int32  [dimensionless]  (detector_id)  [21, 22, ..., 458737, 458739]
  tof                         int32             [ns]  (tof [bin-edge])  [33892, -2147483648]
Data:
                          DataArrayView  [dimensionless]  (detector_id, tof)  [<scipp.DataArray>
Di

In [8]:
scn.instrument_view(data)

### Adding sample position entry

In [9]:
data.coords['sample_position'] = sc.vector(value = np.array([0, 0, 0]), unit = sc.units.m)

In [10]:
data

<scipp.DataArray>
Dimensions: Sizes[detector_id:401408, tof:1, ]
Coordinates:
  detector_id                 int32  [dimensionless]  (detector_id)  [1, 2, ..., 401407, 401408]
  position                  vector3              [m]  (detector_id)  [(0.499023, 0.41833, -4.42986), (0.500969, 0.41833, -4.42986), ..., (0.993148, 1.97683, -4.35336), (0.995093, 1.97683, -4.35336)]
  sample_position           vector3              [m]  ()  [(0, 0, 0)]
  source_position           vector3              [m]  ()  [(0, 0, 25.3)]
  tof                         int32             [ns]  (tof [bin-edge])  [33892, -2147483648]
Data:
                          DataArrayView  [dimensionless]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, ..., <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

, <scipp.DataArray>
Dimensions: Sizes[event:0, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  []
  tof                         int32             [ns]  (event)  []
Data:
                            float32         [counts]  (event)  []  []

]
Attributes:
  end_time                   string  [dimensionless]  ()  ["2021-02-23T08:40:00.000Z"]
  experiment_title           string  [dimensionless]  ()  ["My experiment"]
  monitor_1               DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:118013, tof:1, ]
Coordinates:
  detector_id                 int32  [dimensionless]  (detector_id)  [21, 22, ..., 458737, 458739]
  tof                         int32             [ns]  (tof [bin-edge])  [33892, -2147483648]
Data:
                          DataArrayView  [dimensionless]  (detector_id, tof)  [<scipp.DataArray>
Dimensions: Sizes[event:1, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2021-02-23T09:39:13.789775236]
  tof                         int32             [ns]  (event)  [1337174827]
Data:
                            float32         [counts]  (event)  [1]  [1]

, <scipp.DataArray>
Dimensions: Sizes[event:1, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2021-02-23T09:39:13.789775236]
  tof                         int32             [ns]  (event)  [1521115875]
Data:
                            float32         [counts]  (event)  [1]  [1]

, ..., <scipp.DataArray>
Dimensions: Sizes[event:1, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2021-02-23T09:39:13.789775236]
  tof                         int32             [ns]  (event)  [2147483647]
Data:
                            float32         [counts]  (event)  [1]  [1]

, <scipp.DataArray>
Dimensions: Sizes[event:1, ]
Coordinates:
  pulse_time              datetime64             [ns]  (event)  [2021-02-23T09:39:13.789775236]
  tof                         int32             [ns]  (event)  [178208161]
Data:
                            float32         [counts]  (event)  [1]  [1]

]

]
  monitor_2               DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: Sizes[detector_id:118013, tof:1, ]
Coordinates:
  detector_id                 int32  [dimensionless]  (detector_id)  [21, 22, ..., 458737, 458739]
  tof                         int32             [ns]  (tof [bin-edge])  [33892, -2147483648]
Data:
                          Dat

In [11]:
scn.instrument_view(data)

In [12]:
data.bins.sum()["tof", 0].plot()

In [13]:
data.bins.sum().plot()

In [14]:
data_wave_event = scn.convert(data, 'tof', 'wavelength', scatter='True')

### This is not working 

In [15]:
scn.load(filename=f'{path}/2021-02-23_0939_with_monitors.nxs', mantid_alg='LoadEventNexus')

FrameworkManager-[Notice] Welcome to Mantid 6.0.20210412.848
FrameworkManager-[Notice] Please cite: http://dx.doi.org/10.1016/j.nima.2014.07.029 and this release: http://dx.doi.org/10.5286/Software/Mantid
CheckMantidVersion-[Notice] A new version of Mantid(6.3.0) is available for download from https://download.mantidproject.org
DownloadInstrument-[Notice] All instrument definitions up to date
LoadEventNexus-[Notice] LoadEventNexus started
Run-[Warning] proton_charge log was not a time series property. The value of the total proton charge has not been set
Run-[Warning] gd_prtn_chrg log was not found. Proton Charge set to 0.0
LoadEventNexus-[Warning] Empty proton_charge sample log. You will not be able to filter by time.
LogParser-[Warning] Cannot process ICPevent log. Period 1 assumed for all data.
LoadInstrument-[Warning] The instrument definition filename does not contain _Definition. Your instrument name will be set to: 2021-02-23_0939_with_monitors.nxs
LoadInstrument-[Error] Group::

RuntimeError: LoadEventNexus-v1: unidentifiable C++ exception